# PARSING статей про баню - Набор СКРИПТОВ для работы в автономном режиме (сохранённые HTML коды)

In [2]:
# Импортируем нужные нам библиотеки.
import pandas as pd
from io import StringIO
import numpy as np

In [7]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеки для парсинга сайтов !!!!!!!!!!!!!
import requests
import lxml
import os
import json
import csv

# Работа с файлами XLSX
import openpyxl
import xlsxwriter

# Использование объекта datetime :
from datetime import datetime
from time import sleep
import time
import random

In [2]:
# Выполним импорты:
# from gspread import Client, Spreadsheet, Worksheet, service_account
from gspread import Client, Spreadsheet, Worksheet, service_account, exceptions
from typing import List, Dict

### Путь к папкам и файлам с данными - Path to folders and files with data

#### ВНЕШНИЕ (скачанные) данные: ../data/external/(csv, html, json, xls)/file_name.*
#### ВРЕМЕННЫЕ данные: ../data/interim/(csv, html, json, xls)/file_name.*
#### ФИНАЛЬНЫЕ данные: ../data/processed/(csv, html, json, xls)/file_name.*

## Константы для проекта

# ФУНКЦИЯ № 1. Определение переменных

In [3]:
def variable():
# 0. Создаём словарь заголовков
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    
# 1. Список адресов основного хоста
    hosts = ['https://chelyabinsk.bani.ru.com',
             'https://svoy-bany.ru'
             ]
    
# 2. Список адресов страниц для парсинга
    urls = ['https://chelyabinsk.bani.ru.com/poleznye-statiy/',
            'https://svoy-bany.ru/article/'
            ]

# 3. Список имен сайтов
    names = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
    # names = []
    # for u in urls:
    #     names.append({
    #         'name': u.split('//')[1].split('.')[0]
    #         })
    
# 4. Словарь путей для папок с данными
    type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xls']
    path_files = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xls/'
                ]
    extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xls']
    
    paths = [{'type_file': type_files[i], 'path_file': path_files[i], 'extension_file': extension_files[i]} for i in range(len(type_files))]
    
    # paths = []
    # for i in range(len(type_files)):
    #     paths.append({
    #         'type_file': type_files[i],
    #         'path_file': path_files[i],
    #         'extension_file': extension_files[i]
    #         })
    

    return headers, hosts, urls, names, paths

In [10]:
for i in range(5):
    print(type(variable()[i]))

<class 'dict'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


#### Доступ к элементам словаря из списка словарей
Мы можем получить доступ к элементам в списке словарей, используя индексацию для словаря и ключи для определенных значений внутри словаря.

In [213]:
# К элементам "ПУТЬ К ФАЙЛУ" - "PATHS"
variable()[4][0]

{'type_file': 'csv',
 'path_file': '../data/processed/csv/',
 'extension_file': '.csv'}

In [13]:
# К элементам "ПУТЬ К ФАЙЛУ" - "PATHS"
variable()[4][0]['type_file']

'csv'

In [15]:
# К элементам "ПУТЬ К ФАЙЛУ" - "PATHS"
variable()[4][0]['path_file']

'../data/processed/csv/'

In [7]:
# Тип переменной для "PATHS"
type(variable()[4][0])

dict

In [212]:
# К элементам "ИМЯ ФАЙЛА" - "FILES"
for key, value in variable()[3][0].items():
    print(key, value)

name chelyabinsk


In [9]:
# Доступ к элементам словаря "NAMES" по ключу "NAME"
variable()[3][1]['name']

'svoy-bany'

In [12]:
names_list = variable()[3]
print(type(names_list))

for nl in names_list:
    print(nl, type(nl))
    print(nl.items(), nl.items())
    for key, value in nl.items():
        print(f'{key} - {value}')

<class 'list'>
{'name': 'chelyabinsk'} <class 'dict'>
dict_items([('name', 'chelyabinsk')]) dict_items([('name', 'chelyabinsk')])
name - chelyabinsk
{'name': 'svoy-bany'} <class 'dict'>
dict_items([('name', 'svoy-bany')]) dict_items([('name', 'svoy-bany')])
name - svoy-bany


In [15]:
for nl in names_list:
    print([*nl][0])
    for key, value in nl.items():
        print(f'{key} - {value}')

name
name - chelyabinsk
name
name - svoy-bany


In [16]:
mydict = {'a': 'one', 'b': 'two', 'c': 'three'}
mykeys = [*mydict]          #list of keys
myvals = [*mydict.values()] #list of values

print(mykeys, mykeys[0])
print(myvals, myvals[0])

['a', 'b', 'c'] a
['one', 'two', 'three'] one


In [60]:
type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xls']
files_extension = ['.csv', '.html', '.jpg', '.json', '.txt', '.xls']
files_path = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xls/'
                ]
paths = []
for i in range(len(type_files)):
    paths.append({
        'type_file': type_files[i],
        'file_path': files_path[i],
        'file_extension': files_extension[i]
        })

# paths = [{'type_file': type_files[i], 'file_path': files_path[i], 'file_extension': files_extension[i]} for i in range(len(names))]
print(paths)

[{'type_file': 'csv', 'file_path': '../data/processed/csv/', 'file_extension': '.csv'}, {'type_file': 'html', 'file_path': '../data/interim/html/', 'file_extension': '.html'}, {'type_file': 'jpg', 'file_path': '../data/processed/img/', 'file_extension': '.jpg'}, {'type_file': 'json', 'file_path': '../data/processed/json/', 'file_extension': '.json'}, {'type_file': 'txt', 'file_path': '../data/processed/txt/', 'file_extension': '.txt'}, {'type_file': 'xls', 'file_path': '../data/processed/xls/', 'file_extension': '.xls'}]


In [61]:
paths = [{'type_file': type_files[i], 'file_path': files_path[i], 'file_extension': files_extension[i]} for i in range(len(type_files))]
print(paths)

[{'type_file': 'csv', 'file_path': '../data/processed/csv/', 'file_extension': '.csv'}, {'type_file': 'html', 'file_path': '../data/interim/html/', 'file_extension': '.html'}, {'type_file': 'jpg', 'file_path': '../data/processed/img/', 'file_extension': '.jpg'}, {'type_file': 'json', 'file_path': '../data/processed/json/', 'file_extension': '.json'}, {'type_file': 'txt', 'file_path': '../data/processed/txt/', 'file_extension': '.txt'}, {'type_file': 'xls', 'file_path': '../data/processed/xls/', 'file_extension': '.xls'}]


In [18]:
# d0 = paths[0]
# print(d0)  
# print(paths[0]['type_file'])

#### Создание списка словарей из списков
Если у нас уже есть список данных, и мы хотим превратить его в список словарей. Мы можем использовать цикл или списочное включение, чтобы сделать это.

In [34]:
# Создание списка словарей из другого списка
# Creating a list of dictionaries from another list

names = ["Alice", "Bob", "Charlie"]
ages = [25, 30, 22]

a = [{"name": names[i], "age": ages[i]} for i in range(len(names))]
print(a)

[{'name': 'Alice', 'age': 25}, {'name': 'Bob', 'age': 30}, {'name': 'Charlie', 'age': 22}]


In [53]:
a=[]
for i in range(len(names)):
    
    a.append({
        'name': names[i],
        'age': ages[i]
    })
print(a)

[{'name': 'Alice', 'age': 25}, {'name': 'Bob', 'age': 30}, {'name': 'Charlie', 'age': 22}]


In [45]:
print(a)

{'name': 'Charlie', 'age': 22}


#### Доступ к элементам словаря из списка словарей
Мы можем получить доступ к элементам в списке словарей, используя индексацию для словаря и ключи для определенных значений внутри словаря.

In [37]:
# Доступ к элементам словаря из списка словарей
lod = [
    {'name': 'Alice', 'age': 25, 'city': 'New York'},
    {'name': 'Bob', 'age': 30, 'city': 'Los Angeles'},
    {'name': 'Charlie', 'age': 35, 'city': 'Chicago'}
]

# Access the second dictionary in the list
d1 = lod[1]
print(d1)  
print(lod[0]['name'])

{'name': 'Bob', 'age': 30, 'city': 'Los Angeles'}
Alice


In [51]:
# Прописываем путь и характеристики файла
# 1. Список адресов основного хоста
hosts = ['https://chelyabinsk.bani.ru.com',
         'https://svoy-bany.ru'
        ]

# 2. Список адресов страниц для парсинга
urls = ['https://chelyabinsk.bani.ru.com/poleznye-statiy/',
        'https://svoy-bany.ru/article/'
           ]
# 3. Создаём словарь заголовков
# Это нужно для некоторых сайтов (но не для всех - для этого необязательно)
# чтобы показать сайту, что ВЫ НЕ БОТ, а обычный пользователь
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

# 4. Пути для директорий с файлами
paths = ['../data/processed/csv/',
        '../data/processed/img/',
        '../data/processed/json/',
        '../data/processed/txt/',
        '../data/processed/xls/',
        '../data/interim/html/'
       ]

# 5. Переменные для файлов
file_start = 'barrels_articles_'
file_extension = ['.csv', '.png', '.json', '.txt', '.xlsx', '.html']
# FILE = paths + file_start + file_name + file_extension
# print(f'data: {paths[2]} -- type: {type(paths[2])}')

for f in hosts:
    file_name = f.split('//')[1].split('.')[0]
    sheet_name = file_name
    print(paths[2] + file_start + file_name + file_extension[2])

../data/processed/json/barrels_articles_chelyabinsk.json
../data/processed/json/barrels_articles_svoy-bany.json


In [216]:
urls = ['https://chelyabinsk.bani.ru.com/poleznye-statiy/',
            'https://svoy-bany.ru/article/'
            ]

# 4. Список имен сайтов
# names = []
# for u in urls:
#     names.append({
#         'name': u.split('//')[1].split('.')[0]
#         })

names = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
print(names)
print(*names)
# print(*names.values())

[{'name': 'chelyabinsk'}, {'name': 'svoy-bany'}]
{'name': 'chelyabinsk'} {'name': 'svoy-bany'}


In [217]:
# Преобразование словаря в список. Работа со словарём: ключи и значения.
test = {'foo': 'bar', 'hello': 'world'}
print(list(test))
# ['foo', 'hello']
print(list(test)[0])
# 'foo'

mydict = {'a': 'one', 'b': 'two', 'c': 'three'}
mykeys = [*mydict]          #list of keys
myvals = [*mydict.values()] #list of values

print(mykeys, mykeys[0])
print(myvals, myvals[0])

['foo', 'hello']
foo
['a', 'b', 'c'] a
['one', 'two', 'three'] one


## GET-запросы на сайты для прочтения HTML кода и сохранения его в переменную

# ФУНКЦИЯ № 2. Сохранение HTML-кода для тестирования

In [66]:
# import os
# Отправляем запрос на страницу, передаём в качестве GET-параметра URL-адрес
# Сохраним нашу страницу в файл И ЗАКОММЕНТИРУЕМ ЭТУ КОМАНДУ за ненадобностью

def write_html(headers, url, path):
    # path = '../data/interim/html/'
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path[1]['path_file']):
        print(f'Folder "{path[1]['path_file']}" does NOT exist - it will be created !!!')
        os.mkdir(path[1]['path_file'])
    else:
        print(f'Folder "{path[1]['path_file']}" FOUND!!! It`s OK')
    
    print(f'START of function "WRITE_HTML()"\n')
    
    count = 1
# Проходим циклом по списку со ссылками на страницы сайтов
    for u in url:
        r = requests.get(url=u, headers=headers)
        src = r.text

        file_start = 'barrels_articles_'
        name = u.split('//')[1].split('.')[0]
        file_name = file_start + name
        file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']
        print(f'{count}. Function "WRITE_HTML({name})" EXECUTING ...')

# Проверка существования старых файлов с таким же названием (если они существуют - обновляем)
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[1]['path_file']}\" - It will be CREATED.")
        else:
            print(f"\tIn the folder \"{path[1]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
            os.remove(file_path)
            
# Сохраняем код по указанному пути
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(src)
        print(f'\tHTML-code \"{file_name}\" is SAVED in folder \"{path[1]['path_file']}\"')
        print(f'Function "WRITE_HTML({name.upper()})" is EXECUTED\n')
        count += 1

    print(f'Function "WRITE_HTML()" COMPLETED successfully!')

# Безусловный запуск функции
if __name__ == '__main__':
    write_html(variable()[0], variable()[2], variable()[4])

Folder "../data/interim/html/" FOUND!!! It`s OK
START of function "WRITE_HTML()"

1. Function "WRITE_HTML(chelyabinsk)" EXECUTING ...
	In the folder "../data/interim/html/" the file "barrels_articles_chelyabinsk" EXISTS.
	The file "barrels_articles_chelyabinsk" will be DELETED and REWRITTEN.
	HTML-code "barrels_articles_chelyabinsk" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(CHELYABINSK)" is EXECUTED

2. Function "WRITE_HTML(svoy-bany)" EXECUTING ...
	In the folder "../data/interim/html/" the file "barrels_articles_svoy-bany" EXISTS.
	The file "barrels_articles_svoy-bany" will be DELETED and REWRITTEN.
	HTML-code "barrels_articles_svoy-bany" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(SVOY-BANY)" is EXECUTED

Function "WRITE_HTML()" COMPLETED successfully!


# СОХРАНИЛИ ФАЙЛЫ В НУЖНОЙ ПАПКЕ !!!!

## Теперь открыли файлы списком

# ФУНКЦИЯ № 3.0. Чтение HTML-кода, сохранение его в список

In [78]:
# Теперь откроем наши файлы, прочитаем и сохраним коды страниц в СПИСОК: READ_HTML_0

def read_html_0(names, path):
    # path = '../data/interim/html/'
    count = 1
    src = []
    print(f'START of function "READ_HTML()"')
    
    for name in names:
        file_start = 'barrels_articles_'
        name = name['name']
        file_name = file_start + name
        file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
        
        with open(file_path, encoding='utf-8') as file:
            src.append(file.read())
        print(f'\t{count}. HTML-code "{file_name}" is OPENED from {file_path}')
        
        count += 1
    
    print(f'Function "READ_HTML()" COMPLETED successfully!')

    return src

# if __name__ == '__main__':
#     read_html_0(variable()[3], variable()[4])

In [79]:
src = read_html_0(variable()[3], variable()[4][1])
# print(src[0])

START of function "READ_HTML()"
	1. HTML-code "barrels_articles_chelyabinsk" is OPENED from ../data/interim/html/barrels_articles_chelyabinsk.html
	2. HTML-code "barrels_articles_svoy-bany" is OPENED from ../data/interim/html/barrels_articles_svoy-bany.html
Function "READ_HTML()" COMPLETED successfully!


In [80]:
print(src[1])



<!DOCTYPE html>
<html lang="ru">
	<head>
		<!-- META -->
		<meta name="format-detection" content="telephone=no">
		
		<!-- FAVICONS ICON -->
		<link rel="shortcut icon" type="image/x-icon" href="/favicon.ico" />
		<link rel="icon" href="/images/favicon.ico">
		<link rel="shortcut icon" href="/images/favicon.ico" />
		
		<!-- PAGE TITLE HERE -->
		<title>Полезно знать</title>
		<meta property="og:image" content="/bitrix/templates/svoy-bany/images/logo.png" />
		<meta name="cmsmagazine" content="eca93abe1f826dfcb15a2970177f1bf3" />
		<meta name="cmsmagazine" content="788f743ee7952a16864d8025ac8aee05" /><meta name="ktoprodvinul" content="f6d77d9a598006e7" />
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta name="robots" content="index, follow" />
<meta name="keywords" content="баня-бочка, мобильные бани из бруса, готовые бани, гриль-домик, купель, детские площадки для дачи, молодильные чаны" />
<meta name="description" content="Все о банных процедурах и отды

In [65]:
# Преобразование словаря в список. Работа со словарём: ключи и значения.
test = {'foo': 'bar', 'hello': 'world'}
print(list(test))
# ['foo', 'hello']
print(list(test)[0])
# 'foo'

mydict = {'a': 'one', 'b': 'two', 'c': 'three'}
mykeys = [*mydict]          #list of keys
myvals = [*mydict.values()] #list of values

print(mykeys, mykeys[0])
print(myvals, myvals[0])

['foo', 'hello']
foo
['a', 'b', 'c'] a
['one', 'two', 'three'] one


In [ ]:
# file = os.path.join(path, file_name)
# path = os.path.join(location, file) + extension

## Пишем новую функцию для чтения нашего HTML-файла, прочитаем и сохраним коды страниц в список:

# ФУНКЦИЯ № 3. Чтение HTML-кода, сохранение его в список

In [141]:
# Пишем новую функцию для чтения нашего HTML-файла, прочитаем и сохраним коды страниц в список:
def read_html(name, path):
    print(f'START of function "READ_HTML()"')
    count = 1
    
    # print(name)
    file_start = 'barrels_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'{count}. Function "READ_HTML({name.upper()})" is EXECUTING...')
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['path_file']}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['path_file']}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    count += 1
    print(f'"READ_HTML({name.upper()})" COMPLETED SUCCESSFULLY!')

    # return src
    # yield src

# ПОТОМ НУЖНО СДЕЛАТЬ ВМЕСТО RETURN --- YIELD !!!!!!

In [144]:
read_html(variable()[3][0]['name'], variable()[4][1])

START of function "READ_HTML()"
1. Function "READ_HTML(CHELYABINSK)" is EXECUTING...
	In the folder "../data/interim/html/" the file "barrels_articles_chelyabinsk" EXISTS.
	HTML-code "barrels_articles_chelyabinsk" is OPENED from ../data/interim/html/barrels_articles_chelyabinsk.html
"READ_HTML(CHELYABINSK)" COMPLETED SUCCESSFULLY!


In [136]:
print(variable()[3][0]['name'], variable()[4][1])

chelyabinsk {'type_file': 'html', 'path_file': '../data/interim/html/', 'extension_file': '.html'}


In [143]:
print(src)

NameError: name 'src' is not defined

In [357]:
# src = read_html(urls)
# print(f'Number of elements in the list: {len(src)}')
# src = read_html('https://chelyabinsk.bani.ru.com/poleznye-statiy/')

1. Function "READ_HTML(chelyabinsk)" is EXECUTED
	In the folder "../data/interim/html/" the file "barrels_articles_chelyabinsk" EXISTS.
	HTML-code "barrels_articles_chelyabinsk" is OPENED from ../data/interim/html/barrels_articles_chelyabinsk.html



# ФУНКЦИЯ № 4. Определение блоков для парсинга

# ФУНКЦИЯ № 5. Определение имени

In [87]:
def definition_name_list(url):
    names = []
    for u in url:
        names.append(u.split('//')[1].split('.')[0])
    return names

In [88]:
definition_name_list(variable()[1])

['chelyabinsk', 'svoy-bany']

In [99]:
def definition_name_dict(names):
    names_list = []
    for name in names:
        names_list.append(name['name'])
    return names_list

In [100]:
definition_name_dict(variable()[3])

['chelyabinsk', 'svoy-bany']

In [103]:
names = variable()[3]
for name in names:
    print(name['name'])

chelyabinsk
svoy-bany


# ФУНКЦИЯ № 5.1.OOP. Проверка наличия папки для файлов CSV

In [4]:
def checking_folder(path):
    # path = '../data/processed/csv/' = variable()[4][0]['path_file']
    
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path['path_file']):
        try:
            print(f"Folder \"{path['type_file'].upper()}\" does NOT exist - it will be CREATED !!!\n")
            os.mkdir(path)
        except KeyError as e:
            print(f"Sorry, but you have the folder name WRONG! There CAN't BE a folder \"{path}\"! % str(e)")
    else:
        print(f"Folder \"{path['type_file'].upper()}\" FOUND!!! It`s OK\n")

    # return path

In [205]:
checking_folder(variable()[4][0])
# checking_folder('../data/processed/csv/')

Folder "CSV" FOUND!!! It`s OK



# ФУНКЦИЯ № 5.2. Проверка наличия старых CSV файлов в папке, создание заголовков

In [120]:
# Для единичной проверки - def checking_file_csv(path)
def checking_file_csv(path):

    file_start = 'barrels_articles_'
    names = variable()[3]
    print(f'START of function "CHECKING_FILE_CSV()"')
    count = 1

    for name in names:
        name = name['name']
        file_name = file_start + name
        file_path = os.path.join(path[0]['path_file'], file_name) + path[0]['extension_file']
        print(f"\t{count}.START function CHECKING_FILE_CSV(\"{name.upper()}\")")
        
    # Проверка существования старых CSV файлов с таким же названием (если они существуют - обновляем).
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[0]['path_file']}\" - It will be CREATED.\n")
        else:
            print(f"\tIn the folder \"{path[0]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.\n')
            # os.remove(file_path) # DON'T FORGET TO OPEN THE LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!
            
    # Собираем заголовки для CSV-файла
        with open(file_path, 'w', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(
                (
                    'Number',
                    'Topic',
                    'Link',
                    'DateTime'
                )
            )
        count += 1
    
    print(f'Function "CHECKING_FILE_CSV()" COMPLETED successfully!')

In [121]:
checking_file_csv(variable()[4])

START of function "CHECKING_FILE_CSV()"
	1.START function CHECKING_FILE_CSV("CHELYABINSK")
	In the folder "../data/processed/csv/" the file "barrels_articles_chelyabinsk" EXISTS.
	The file "barrels_articles_chelyabinsk" will be DELETED and REWRITTEN.

	2.START function CHECKING_FILE_CSV("SVOY-BANY")
	In the folder "../data/processed/csv/" the file "barrels_articles_svoy-bany" EXISTS.
	The file "barrels_articles_svoy-bany" will be DELETED and REWRITTEN.

Function "CHECKING_FILE_CSV()" COMPLETED successfully!


In [5]:
# Для подготовки CSV файла в YIELD функции - def csv_preparation(path, name)
def csv_preparation(path, name):
    
    print(f'\tSTART "CSV_PREPARATION({name.upper()})"')
    
    file_start = 'barrels_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
        
# Проверка существования старых CSV файлов с таким же названием (если они существуют - обновляем).
    if not os.path.exists(file_path):
        print(f"\t\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.\n")
    else:
        print(f"\t\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        print(f'\t\tThe file "{file_name}" will be DELETED and REWRITTEN.')
        # os.remove(file_path) # DON'T FORGET TO OPEN THE LINE !!!! - Это необязательно, потому что - "W"
        
# Собираем заголовки для CSV-файла
    with open(file_path, 'w', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                'Number',
                'Topic',
                'Link',
                'DateTime'
            )
        )
    
    print(f'\tEND "CSV_PREPARATION({name.upper()})"')

In [199]:
csv_preparation(variable()[4][0], 'chelyabinsk')

	START "CSV_PREPARATION(CHELYABINSK)"
		In the folder "CSV" the file "barrels_articles_chelyabinsk" EXISTS.
		The file "barrels_articles_chelyabinsk" will be DELETED and REWRITTEN.
	END "CSV_PREPARATION(CHELYABINSK)"


# ФУНКЦИЯ № 5.3. Создание CSV-файла

In [6]:
# Для заполнения CSV файла содержимым построчно - def save_csv(name, count, title, link, date_time, path):
def save_csv(name, count, title, link, date_time, path):
    
    file_start = 'barrels_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    
# Заполняем CSV файл построчно (внутри цикла)
    with open(file_path, 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                count,
                title,
                link,
                date_time
            )
        )

In [87]:
# save_csv(urls)

# ФУНКЦИЯ № 6. Создание JSON-файла

In [7]:
def save_json(name, path, articles_list):

# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path['path_file']):
        print(f"Folder \"{path['type_file'].upper()}\" does NOT exist - it will be CREATED !!!")
        os.mkdir(path['path_file'])
    else:
        print(f"Folder \"{path['type_file'].upper()}\" FOUND!!! It`s OK")


    file_start = 'barrels_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'START function SAVE_JSON("{name.upper()}")')
    
# Проверка существования старых CSV файлов с таким же названием (если они существуют - обновляем).
    if not os.path.exists(file_path):
        print(f"\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.")
    else:
        print(f"\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
        os.remove(file_path) # DON'T FORGET TO OPEN THE LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!

    
# Создаём третий файл в формате JSON, в который будем записывать данные в виде словаря
# который весьма полезен и удобен для чтения (в дополнение к формату CSV)
    with open(file_path, 'a', encoding='utf-8') as file:
        json.dump(articles_list, file, indent=4, ensure_ascii=False)

    print(f"FINISH function SAVE_JSON(\"{name.upper()}\")")

# ФУНКЦИЯ № 7. Чтение HTML-кода, БЕЗ сохранения его в список. Команда YIELD.

In [26]:
# Функция для чтения HTML кода из Интернета на-лету (НУЖНО МУДРИТЬ С ПЕРЕМЕННЫМИ)
def read_html_yield(url, path):
# Счётчик для URL-ов:
    count = 1
    
    for u in url:    
        file_start = 'barrels_articles_'
        name = u.split('//')[1].split('.')[0]
        file_name = file_start + name
        file_path = os.path.join(path, file_name) + '.html'
        print(f'{count}. Function READ_HTML("{name.upper()}") is EXECUTING...')
        
# Проверка существования файлов с HTML-кодом.
        if os.path.exists(file_path):
            print(f"\tIn the folder \"{path}\" the file \"{file_name}\" EXISTS.")
            
            with open(file_path, encoding='utf-8') as file:
                src = file.read()
            print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
            
        else:
            print(f"\tThere is NO file \"{file_name}\" in the \"{path}\" folder!")
            print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
            break
        
        count += 1
        print(f'FINAL_HTML - "{name.upper()}"\n')
    
        yield src, name
    print('FINAL_READ')

In [8]:
# Функция для чтения сохранённых HTML-файлов в HTML папке - выбор по имени
def read_html_yield_names(name, path, count):
    print(f'{count}.START of function "READ_HTML_YIELD_NAMES()"')
    # count = 1
    
    file_start = 'barrels_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f"\tFunction \"READ_HTML_YIELD_NAMES({name.upper()})\" is EXECUTING...")
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f"\t\"{path['type_file'].upper()}\"-code \"{file_name}\" is OPENED from \"{file_path}\"")
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['type_file'].upper()}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    # count += 1
    print(f'"READ_HTML_YIELD_NAMES({name.upper()})" COMPLETED SUCCESSFULLY!\n')

    
    # return src
    yield src
    # print('FINAL_READ')

In [161]:
read_html_yield_names(variable()[3][1]['name'], variable()[4][1], 1)

<generator object read_html_yield_names at 0x000001CD955D96D0>

## Парсер - генератор, оптимизируем код

# ФУНКЦИЯ № 8. Определение блоков для парсинга интересующих сайтов (функция YIELD)

In [1]:
def abc():
    a = [1, 2, 3]
    b = ['a', 'b', 'c']
    c = [234, 456, 789]
    return a,b,c

In [2]:
print(abc())

([1, 2, 3], ['a', 'b', 'c'], [234, 456, 789])


In [3]:
a,b,c = abc()
for x, y, z in zip(a, b, c):
    print(x, y, z)

1 a 234
2 b 456
3 c 789


In [8]:
# Каждый раз функция проходит по внутренней ссылке на товар и выполнение цикла приостанавливается благодаря команде !!! "YIELD" !!!
def definition_blocks():
# Определяем переменные с помощью функции VARIABLE()
    hosts = variable()[1]
    urls = variable()[2]
    names = variable()[3]
    
    paths = variable()[4]
    path_csv = paths[0]
    path_html = paths[1]
    path_json = paths[3]
    
    print(f'\t\tSTART of function "DEFINITION_BLOCKS()"\n')
# Проверка наличия папки для файлов
    # checking_folder_csv(paths)
    checking_folder(path_csv)
    count = 1
    for item in range(len(names)):
        
        for src in read_html_yield_names(names[item]['name'], path_html, count):
        
# Определяем объекты BeautifulSoup:
            soup = BeautifulSoup(src, 'lxml')
# Счётчик для ссылок на статьи
            count_url = 1
        
            file_start = 'barrels_articles_'
            name = names[item]['name']
        
# Объявляем начало чтения ссылок
        print(f'\t{item+1}.START of function "DEFINITION_BLOCKS({name.upper()})" and it`s EXECUTING ...')
        # print(f'\tFunction GET_URL("{name.upper()}") is EXECUTING...')
    
        if name == 'chelyabinsk':
# Заполняем заголовки для CSV-файла
            csv_preparation(path_csv, name)
            
            file_name = file_start + name
            articles_list = []
            blocks = soup.find('div', class_='desc list').find_all('a')
            count_link = 1

            print(f'START SAVE_CSV("{name.upper()}"):')
            for item in blocks:
                title = item.text
                link = item.get("href")
                date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'Link №{count_link}. {title}')

                articles_list.append(
                    {
                        "Number": count_link,
                        "Topic": title,
                        "Link": link,
                        "DateTime": date_time
                    }
                )
# Заполняем строки CSV-файла                
                save_csv(name, count_link, title, link, date_time, path_csv)
                count_link += 1
            print(f'\t\"{path_csv['type_file'].upper()}\"-file \"{file_name}\" is SAVED in folder \"{path_csv['type_file'].upper()}\"')
            print(f'END SAVE_CSV - "{name.upper()}"\n')
# Заполняем JSON файл
            save_json(name, path_json, articles_list)
        
        elif name == 'svoy-bany':
# Заполняем заголовки для CSV-файла
            csv_preparation(path_csv, name)

            file_name = file_start + name
            articles_list = []
            blocks = soup.find_all('p', class_= 'news-item')
            count_link = 1
            
            for item in blocks:
                title = item.text.strip()
                link = hosts[1] + item.find('a').get('href')
                date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(f'Link №{count_link}. {title}')
                
                articles_list.append(
                    {
                        "Number": count_link,
                        "Topic": title,
                        "Link": link,
                        "DateTime": date_time
                    }
                )
# Заполняем строки CSV-файла
                save_csv(name, count_link, title, link, date_time, path_csv)
                count_link += 1
            print(f'\t\"{path_csv['type_file'].upper()}\"-file \"{file_name}\" is SAVED in folder \"{path_csv['type_file'].upper()}\"')
            print(f'FINAL SAVE_CSV - "{name.upper()}"\n')
# Заполняем JSON файл
            save_json(name, path_json, articles_list)
        
        # sleep(random.randrange(2, 4)) # Для симуляции работы реального пользователя, а не робота
        
        count_url += 1
        count += 1
        
        print(f'"DEFINITION_..({name.upper()})" COMPLETED SUCCESSFULLY!\n')
    print(f'\t\tEND of function "DEFINITION_BLOCKS()"')
    # return name

# Безусловный запуск функции
if __name__ == '__main__':
    definition_blocks()

		START of function "DEFINITION_BLOCKS()"

Folder "CSV" FOUND!!! It`s OK

1.START of function "READ_HTML_YIELD_NAMES()"
	Function "READ_HTML_YIELD_NAMES(CHELYABINSK)" is EXECUTING...
	In the folder "HTML" the file "barrels_articles_chelyabinsk" EXISTS.
	"HTML"-code "barrels_articles_chelyabinsk" is OPENED from "../data/interim/html/barrels_articles_chelyabinsk.html"
"READ_HTML_YIELD_NAMES(CHELYABINSK)" COMPLETED SUCCESSFULLY!

	1.START of function "DEFINITION_BLOCKS(CHELYABINSK)" and it`s EXECUTING ...
	START "CSV_PREPARATION(CHELYABINSK)"
		In the folder "CSV" the file "barrels_articles_chelyabinsk" EXISTS.
		The file "barrels_articles_chelyabinsk" will be DELETED and REWRITTEN.
	END "CSV_PREPARATION(CHELYABINSK)"
START SAVE_CSV("CHELYABINSK"):
Link №1. Как сделать фундамент для бани-бочки?
Link №2. Печь в бане-бочке: печь с топкой через перегородку или на улицу — плюсы и минусы
Link №3. Правила эксплуатации бань-бочек
Link №4. Что нужно знать при покупке?
Link №5. Эксплуатация бани-б

In [11]:
# definition_blocks()

In [ ]:
# BeautifulSoup(str(f.readlines()), "html.parser")

In [ ]:
## Создание класса "Article"

In [76]:
obj_list = []
for i in range(len(names_l)):
    print(names_l[i])
    if names_l[i] == 'chelyabinsk':
        obj = Chel(names_l[i])
    elif names_l[i] == 'svoy-bany':
        obj = Svoy(names_l[i])
    obj_list.append(obj)
print(obj_list)

chelyabinsk
svoy-bany
[<__main__.Chel object at 0x000001CD92D94410>, <__main__.Svoy object at 0x000001CD935DFA10>]


In [78]:
print(obj_list[0].name, obj_list[0].link)

chelyabinsk link_ch


In [ ]:
if -10 < a < 10:
    obj = One(a)
else:
    obj = Two(a)

In [16]:
# 1. Список адресов основного хоста
hosts = [
    'https://chelyabinsk.bani.ru.com',
    'https://svoy-bany.ru'
    ]
    
# 2. Список адресов страниц для парсинга
urls = [
    'https://chelyabinsk.bani.ru.com/poleznye-statiy/',
    'https://svoy-bany.ru/article/'
    ]

# 3. Список имен сайтов
names_d = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
names_l = [u.split('//')[1].split('.')[0] for u in urls]

# 4. Словарь для блоков, заголовков, ссылок
blocks = [
    "soup.find('div', class_='desc list').find_all('a')",
    "soup.find_all('p', class_= 'news-item')"
    ]

titles = [
    "item.text",
    "item.text.strip()"
    ]

links = [
    "item.get('href')",
    "hosts[1] + item.find('a').get('href')"
    ]
    
elements = [{'host': hosts[i], 'url': urls[i], 'block': blocks[i], 'title': titles[i], 'link': links[i]} for i in range(len(urls))]
print(elements[0])
# paths
print()
print(names_d)
print(names_l)
print()


# 5. Словарь атрибутов Супа
divs = ['div', 'p']
classes = ['desc list', 'news-item']
# extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xls']

patt = [{'div': divs[i], 'class': classes[i]} for i in range(len(divs))]
print(patt, type(patt))
print(patt[0], type(patt[0]))
patt[0]['class']

{'host': 'https://chelyabinsk.bani.ru.com', 'url': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/', 'block': "soup.find('div', class_='desc list').find_all('a')", 'title': 'item.text', 'link': "item.get('href')"}

[{'name': 'chelyabinsk'}, {'name': 'svoy-bany'}]
['chelyabinsk', 'svoy-bany']

[{'div': 'div', 'class': 'desc list'}, {'div': 'p', 'class': 'news-item'}] <class 'list'>
{'div': 'div', 'class': 'desc list'} <class 'dict'>


'desc list'

In [41]:
print(elements[0]['url'])
print(names_d[0]['name'])
print(names_l[0])

https://chelyabinsk.bani.ru.com/poleznye-statiy/
chelyabinsk
chelyabinsk


In [75]:
article1 = Article('Tom', "soup.find('div', class_='desc list').find_all('a')", "i.get('href')")

Article 'TOM': Block - soup.find('div', class_='desc list').find_all('a'); Link - i.get('href').


In [17]:
paths = variable()[3]
print(type(paths))
print(paths['csv'])

<class 'dict'>
../data/processed/csv/


In [18]:
path_csv = variable()[3]['csv']
print(path_csv)

../data/processed/csv/


# ФУНКЦИЯ № 10. Собираем все рабочие функции в одну главную с использованием ООП

In [37]:
# STARTING OPTION Class Article - It`s a working option
class Article:
    html = "<div class='findme'> blabla <p> p-tag content</p></div>"
    sources = {'source': 'div, findme'}
    # tag = sources['source'].split(', ')
    
    # sources = {'source1': 'p, class_=\'findme\'',
    def __init__(self, name='NoName', src=html):
        self.name = name
        self.src = src
        self.sources = {'source': 'div, findme'}
        self.tag = self.sources['source'].split(', ')
        self.soup = BeautifulSoup(self.src, 'lxml')
        self.block = self.soup.find(self.tag[0], class_=self.tag[1])
        self.title = self.block.text
    
    def __str__(self):
        # return f"\nArticle '{self.name.upper()}':\nBlock - {self.block}"
        return f"\nArticle '{self.name.upper()}':\nBlock - {self.block}\nTitle - {self.title}"
        # return f"\nArticle:\nBlock - {self.block}\nTitle - {self.title}"

class Chel(Article):
    def __init__(self, name='Some Name', newsrc=Article.html, newsources=Article.sources):
        self.name = name
        self.sources = newsources
        self.src = newsrc
        self.soup = BeautifulSoup(self.src, 'lxml')
        self.tag = self.sources['source'].split(', ')
        self.block = self.soup.find(self.tag[0], class_=self.tag[1])
        self.title = self.block.text
        # self.link = self.block.find('a').get('href')

    def __str__(self):
        # return f"\nChel '{self.name.upper()}':\nBlock - {self.block}\nTitle - {self.title}\nLink - {self.link}"
        return f"\nChel '{self.name.upper()}':\nBlock - {self.block}\nTitle - {self.title}"


# class Svoy(Article):
#     def __init__(self, name, src=None):
#         self.soup = Article(src)
#         self.name = name
#         self.block = self.soup.find_all('p', class_= 'news-item')
#         self.title = self.block.text.strip()
#         self.link = self.block.find('a').get('href')

In [41]:
# Variant 1
obj_main = Article()
# obj_main = Article('chelyabinsk')
print(obj_main)
# print()
b = obj_main.block
# print('Block:', b, '\n')

# Variant 2
block_chel = '''
<div> blabla 
<p class='findmeee'> p-tag content
<a href="https://chelyabinsk.bani.ru.com/poleznye-statiy/kak-sdelat-fundament-dlya-bani-bochki/">Как сделать фундамент для бани-бочки?</a>
</p>
</div>
'''
class_chel = {'source': 'p, findmeee'}
# obj_chel = Chel(newsrc=block_chel, newsources=class_chel)
# obj_chel = Chel('chelyabinsk')
obj_chel = Chel()
print(obj_chel)
# print()
b = obj_chel.block
# print('Block:', b, '\n')


Article 'NONAME':
Block - <div class="findme"> blabla <p> p-tag content</p></div>
Title -  blabla  p-tag content

Chel 'SOME NAME':
Block - <div class="findme"> blabla <p> p-tag content</p></div>
Title -  blabla  p-tag content


In [24]:
# site1 = Chel('chlb', 'block', 'title', 'link')
# site1 = Chel(names_l[0])
# site1 = Chel('chelyabinsk', read_html_yield_names(variable()[3][1]['name'], variable()[4][1], 1))
obj = Chel('chelyabinsk')

# site2 = Svoy(names_l[1])
# soup_test = Article(src)
print(obj)
print()
b = obj.block
print(b)


# obj = Chel(name)


Chel 'CHELYABINSK':
Block - <div class="findme"> blabla <p> p-tag content</p></div>;
Title -  blabla  p-tag content.

<div class="findme"> blabla <p> p-tag content</p></div>


In [11]:
# Каждый раз функция проходит по внутренней ссылке на товар и выполнение цикла приостанавливается благодаря команде !!! "YIELD" !!!
def definition_blocks_class():
# Определяем переменные с помощью функции VARIABLE()
    hosts = variable()[1]
    urls = variable()[2]
    names = variable()[3]
    
    paths = variable()[4]
    path_csv = paths[0]
    path_html = paths[1]
    path_json = paths[3]
    
    print(f'\t\tSTART of function "DEFINITION_BLOCKS()"\n')
# Проверка наличия папки для файлов
    # checking_folder_csv(paths)
    checking_folder(path_csv)
    count = 1
    # for item in range(len(names)):
    for item in range(1):
        for src in read_html_yield_names(names[item]['name'], path_html, count):
        
# Определяем объекты BeautifulSoup:
            # soup = Article(src)
            class_is_multi= { '*' : 'class'}
            soup = BeautifulSoup(src, 'lxml')
            # soup = BeautifulSoup(src, 'lxml', multi_valued_attributes=class_is_multi)
            print("-------------------------------------------------------------------------")
            # print(soup.a['class'])
            
            print('1. src -', type(src))
            print('2. soup -', type(soup))
            print("-------------------------------------------------------------------------")
            
            
# Счётчик для ссылок на статьи
            count_url = 1
        
            file_start = 'barrels_articles_'
            name = names[item]['name']
        
# Объявляем начало чтения ссылок
        print(f'\t{item+1}.START of function "DEFINITION_BLOCKS({name.upper()})" and it`s EXECUTING ...')
        # print(f'\tFunction GET_URL("{name.upper()}") is EXECUTING...')

# -------------------------------------------------------------------------
        if name == 'chelyabinsk':
# Заполняем заголовки для CSV-файла
            # obj = Chel(name, src)
            print("-------------------------------------------------------------------------")
            # print('3. obj.name -', obj.name, '-', type(obj.name))
            print('4. name', name, '-', type(name))
            print("-------------------------------------------------------------------------")
#             csv_preparation(path_csv, name)
            
            file_name = file_start + name
            articles_list = []
            # blocks = soup.find('div', class_='desc list').find_all('a')
            blocks = soup.find(patt[0]['div'], patt[0]['class']).find_all('a')
            # blocks1 = soup.find(b)
            # blocks1 = soup.join(soup, b)
            # blocks1 = BeautifulSoup(blocks1, 'lxml')
            
            print("-------------------------------------------------------------------------")
            print('5. blocks -', blocks, '-', type(blocks))
            # print('6. obj.blocks -', blocks1, type(blocks1))
            print("==========================================================================")
            # for b in blocks:
            #     # b = str(b)
            #     print(b, 'TYPE:', type(b), 'tag_name:', b.name, type(b.name), 'tag.attrs:', b.attrs, type(b.attrs))

            print("==========================================================================")
            
#             # obj.block = soup.find('div', class_='desc list').find_all('a')
#             count_link = 1

#             print(f'START SAVE_CSV("{name.upper()}"):')
#             for item in blocks:
#                 title = item.text
#                 link = item.get("href")
#                 date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 print(f'Link №{count_link}. {title}')

#                 articles_list.append(
#                     {
#                         "Number": count_link,
#                         "Topic": title,
#                         "Link": link,
#                         "DateTime": date_time
#                     }
#                 )
# # Заполняем строки CSV-файла                
#                 save_csv(name, count_link, title, link, date_time, path_csv)
#                 count_link += 1
#             print(f'\t\"{path_csv['type_file'].upper()}\"-file \"{file_name}\" is SAVED in folder \"{path_csv['type_file'].upper()}\"')
#             print(f'END SAVE_CSV - "{name.upper()}"\n')
# # Заполняем JSON файл
#             save_json(name, path_json, articles_list)

# --------------------------------------------------------------------------
        
#         elif name == 'svoy-bany':
# # Заполняем заголовки для CSV-файла
#             csv_preparation(path_csv, name)

#             file_name = file_start + name
#             articles_list = []
#             blocks = soup.find_all('p', class_= 'news-item')
#             count_link = 1
            
#             for item in blocks:
#                 title = item.text.strip()
#                 link = hosts[1] + item.find('a').get('href')
#                 date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 print(f'Link №{count_link}. {title}')
                
#                 articles_list.append(
#                     {
#                         "Number": count_link,
#                         "Topic": title,
#                         "Link": link,
#                         "DateTime": date_time
#                     }
#                 )
# # Заполняем строки CSV-файла
#                 save_csv(name, count_link, title, link, date_time, path_csv)
#                 count_link += 1
#             print(f'\t\"{path_csv['type_file'].upper()}\"-file \"{file_name}\" is SAVED in folder \"{path_csv['type_file'].upper()}\"')
#             print(f'FINAL SAVE_CSV - "{name.upper()}"\n')
# # Заполняем JSON файл
#             save_json(name, path_json, articles_list)
        
        # sleep(random.randrange(2, 4)) # Для симуляции работы реального пользователя, а не робота
        
        count_url += 1
        count += 1
        
        print(f'"DEFINITION_..({name.upper()})" COMPLETED SUCCESSFULLY!\n')
    print(f'\t\tEND of function "DEFINITION_BLOCKS()"')
    # return name

# Безусловный запуск функции
if __name__ == '__main__':
    definition_blocks_class()

		START of function "DEFINITION_BLOCKS()"

Folder "CSV" FOUND!!! It`s OK

1.START of function "READ_HTML_YIELD_NAMES()"
	Function "READ_HTML_YIELD_NAMES(CHELYABINSK)" is EXECUTING...
	In the folder "HTML" the file "barrels_articles_chelyabinsk" EXISTS.
	"HTML"-code "barrels_articles_chelyabinsk" is OPENED from "../data/interim/html/barrels_articles_chelyabinsk.html"
"READ_HTML_YIELD_NAMES(CHELYABINSK)" COMPLETED SUCCESSFULLY!

-------------------------------------------------------------------------
1. src - <class 'str'>
2. soup - <class 'bs4.BeautifulSoup'>
-------------------------------------------------------------------------
	1.START of function "DEFINITION_BLOCKS(CHELYABINSK)" and it`s EXECUTING ...
-------------------------------------------------------------------------
4. name chelyabinsk - <class 'str'>
-------------------------------------------------------------------------
-------------------------------------------------------------------------
5. blocks - [<a href="htt

# FORUM

In [224]:
# Объект NAME класса WEBSITE - это список, состоящий из словарей
names = Website(variable()[1])
print(names.name)

for n in names.name:

# Разбираем список на элементы (всего 2 строки)
    for key, value in n.items():

# Каждый словарь разбираем по ключу и значению (КЛЮЧИ у разных строк одинаковые, а значения РАЗНЫЕ)
        print(key, value)


Folder "../data/processed/csv/" FOUND!!! It`s OK


Folder "../data/processed/csv/" FOUND!!! It`s OK

[{'Name': 'chelyabinsk'}, {'Name': 'svoy-bany'}]
Name chelyabinsk
Name svoy-bany


In [53]:
class_is_multi= { '*' : 'class'}
xml_soup = BeautifulSoup('<p class="body strikeout rty"></p>', 'lxml', multi_valued_attributes=class_is_multi)
xml_soup.p['class']
# print(xml_soup.p['class'])
# ['body', 'strikeout']

['body', 'strikeout', 'rty']

In [169]:
# Variable()[1] - URLS; Variable()[2] - HOSTS
website_list = Website(variable()[1], variable()[2], variable()[3])


Folder "../data/processed/csv/" FOUND!!! It`s OK


Folder "../data/processed/csv/" FOUND!!! It`s OK



In [ ]:
# Просто выполните замену перед разбором :

html = html.replace('class="row bigbox container mi-df-local locked-single"', 'class="row bigbox container mi-df-local single-local"')
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# Обратите внимание, что также было бы возможно (я бы даже сказал предпочтительно ) проанализировать HTML, 
# найти элемент(ы) и изменить атрибуты экземпляра Tag, например:

soup = BeautifulSoup(html, "html.parser")
for elm in soup.select(".row.bigbox.container.mi-df-local.locked-single"):
    elm["class"] = ["row", "bigbox", "container", "mi-df-local", "single-local"]

In [ ]:
# Обратите внимание, что class это специальный многозначный атрибут — вот почему мы устанавливаем значение для списка отдельных классов.
# Демонстрация:
from bs4 import BeautifulSoup

html = """
<div class="row bigbox container mi-df-local locked-single">test</div>
"""

soup = BeautifulSoup(html, "html.parser")
for elm in soup.select(".row.bigbox.container.mi-df-local.locked-single"):
    elm["class"] = ["row", "bigbox", "container", "mi-df-local", "single-local"]

print(soup.prettify())

# Теперь посмотрим, как div обновились классы элементов:

<div class="row bigbox container mi-df-local single-local">
 test
</div>

In [117]:
# 4. Словарь атрибутов Супа
divs_test = ['p', 'span', 'div']
classes_test = ['findme', 'findme2', 'findme33']
# extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xls']

patt_test = [{'class_': classes_test[i]} for i in range(len(divs_test))]
print(patt_test[0], type(patt_test[0]))
patt_test[0]['class_']

{'class_': 'findme'} <class 'dict'>


'findme'

In [72]:
# Привет, мне нужно передать переменную в функцию soup.find(), но она не работает :( Кто-нибудь знает решение этой проблемы?

from bs4 import BeautifulSoup

html = '''<div> blabla
<p class='findme'> p-tag content</p>
</div>'''

sources = {'source1': 'p, findme',
           'source2': '\'span\', class_=\'findme2\'',
           'source3': '\'div\', class_=\'findme3\''}

class_test = 'findme'
tag1 = "p"
test = BeautifulSoup(html, 'lxml')

# this works
#print(test.find('p', class_='findme'))
# >>> <p class="findme"> p-tag content</p>


# this doesn't work
# tag = '\'p\' class_=\'findme\''

tag = sources['source1'].split(', ')
print(sources['source1'], type(sources['source1']))
print(tag, type(tag))
print('tag[0]:', tag[0], type(tag[0]), tag[1], type(tag[1]))

# a source gets passed
print('0.', test.find(tag[0], class_=tag[1]))
print('1.', test.find('p', class_='findme'))
# test.find(sources['source1'])
# >>> None
print('---------------------')
print('!!!')
# print(test.find(divs_test[0], **patt_test[0]))
print('2.', test.find(patt[0]['div'], class_=patt[0]['class']))
test.find(patt[0]['div'], class_= patt[0]['class'])
# obj.block = soup.find('div', class_='desc list')

print('------------')
print('tag1 == tag[0]:', tag1 == tag[0])

p, findme <class 'str'>
['p', 'findme'] <class 'list'>
tag[0]: p <class 'str'> findme <class 'str'>
0. <p class="findme"> p-tag content</p>
1. <p class="findme"> p-tag content</p>
---------------------
!!!
2. None
------------
tag1 == tag[0]: True


In [55]:
# Я пытаюсь разделить это, как предлагается, следующим образом:

pattern = 'p, {"class_": "findme"}'
tag = pattern.split(', ')
tag1 = "p"
print('tag - ', tag, type(tag))
print('tag[0] - ', tag[0], type(tag[0]))
print('tag1 - ', tag1, type(tag1))
print('tag[1] - ', tag[1], type(tag[1]))



# filter_ = dict(tag[1])
filter_ = {'class_': 'findme'}
print('tag[11] - ', filter_, type(filter_))
# date = test.find(tag1, filter)
print('------------')

# Я не получаю ошибок, просто None для даты. Проблема, вероятно, в содержании tag1 и фильтра. Отладчик pycharm выдает мне:
print('tag1 == tag[0]:', tag1 == tag[0])
test.find(tag[0], **filter_)
# tag1 = 'p'
# filter = {'class_': 'findme'}
# test.find(tag1, **filter)
# При печати эти апострофы не отображаются. Можно ли убрать эти апострофы?

tag -  ['p', '{"class_": "findme"}'] <class 'list'>
tag[0] -  p <class 'str'>
tag1 -  p <class 'str'>
tag[1] -  {"class_": "findme"} <class 'str'>
tag[11] -  {'class_': 'findme'} <class 'dict'>
------------
tag1 == tag[0]: True


<p class="findme"> p-tag content</p>

In [49]:
tag1 = 'p'
filter_ = {'class_': 'findme'}
test.find(tag1, **filter_)

<p class="findme"> p-tag content</p>

In [87]:
# Первый аргумент — это имя тега , а ваша строка его не содержит.
# BeautifulSoup (или Python в целом) не будет анализировать такую строку, он не может догадаться,
# что вы поместили в это значение какой-то произвольный синтаксис Python.

# Выделите компоненты:

tag = 'p'
filter = {'class_': 'findme'}
print('tag[1] - ', filter, type(filter),'\n')
test.find(tag, **filter)

tag[1] -  {'class_': 'findme'} <class 'dict'> 



<p class="findme"> p-tag content</p>

In [64]:
# Хорошо, я понял, спасибо еще раз.

# dic_date = {'source1': 'p, class:findme', other sources ...}
dic_date = {'source1': 'p, class:findme'}

pattern = dic_date['source1']
tag = pattern.split(', ')
if len(tag) is 2:
    att = tag[1].split(':')  # getting the attribute
    att = {att[0]: att[1]}  # building a dictionary for the attributes
    date = soup.find(tag[0], att)
else:
    date = soup.find(tag[0])  # if there is only a tag without an attribute

    # Ну, выглядит не очень красиво, но работает :)

<>:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
C:\Users\User\AppData\Local\Temp\ipykernel_8440\3122824207.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if len(tag) is 2:
C:\Users\User\AppData\Local\Temp\ipykernel_8440\3122824207.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if len(tag) is 2:
C:\Users\User\AppData\Local\Temp\ipykernel_8440\3122824207.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if len(tag) is 2:
C:\Users\User\AppData\Local\Temp\ipykernel_8440\3122824207.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if len(tag) is 2:
C:\Users\User\AppData\Local\Temp\ipykernel_8440\3122824207.py:8: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if len(tag) is 2:


NameError: name 'soup' is not defined

In [ ]:
# Test for GIT